# NYC Apartment Search

_[Project prompt](https://docs.google.com/document/d/1BYVyFBDcTywdUlanH0ysfOrNWPgl7UkqXA7NeewTzxA/edit#heading=h.bpxu7uvknnbk)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add code as you wish._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an idea of a possible approach.**_

## Setup

In [83]:
# All import statements needed for the project, for example:

import json
import pathlib
import urllib.parse

import pandas as pd
import geoalchemy2 as gdb
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import requests
import shapely
import sqlalchemy as db
import psycopg2

from sqlalchemy import create_engine
from geoalchemy2 import WKTElement, Geometry
from sqlalchemy.orm import declarative_base

In [45]:
#!pip install GeoAlchemy2

In [6]:
# Any constants you might need; some have been added for you

# Where data files will be read from/written to - this should already exist
DATA_DIR = pathlib.Path("data")
ZIPCODE_DATA_FILE = DATA_DIR / "zipcodes" / "nyc_zipcodes.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

NYC_DATA_APP_TOKEN = "UydgE9GUfZyuG9IpbKml1aKct"
BASE_NYC_DATA_URL = "https://data.cityofnewyork.us/"
NYC_DATA_311 = "erm2-nwe9.geojson"
NYC_DATA_TREES = "5rq2-4hqu.geojson"

DB_NAME = "IE4501_Project_DB"
DB_USER = "postgres"
DB_URL = f"postgresql+psycopg2://{DB_USER}@localhost/{DB_NAME}"
DB_SCHEMA_FILE = "schema.sql"
# directory where DB queries for Part 3 will be saved
QUERY_DIR = pathlib.Path("queries")

In [7]:
# Make sure the QUERY_DIRECTORY exists
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir()

## Part 1: Data Preprocessing

In [77]:
def load_and_clean_zipcodes(zipcode_datafile):
     # Load the data using geopandas
    gdf_zip = gpd.read_file(ZIPCODE_DATA_FILE)
    
    # Drop unnecessary columns. Here, you need to decide which columns are relevant
    # For example, if you only need the zipcode and geometry, you can drop the rest
    columns_to_keep = ['ZIPCODE', 'geometry']
    gdf_zip = gdf_zip[columns_to_keep]

    # Remove invalid data points
    # This depends on what you define as invalid. As an example, you might want to remove rows with missing values
    gdf_zip = gdf_zip.dropna()
    gdf_zip = gdf_zip.drop_duplicates(subset = 'ZIPCODE')
    gdf_zip = gdf_zip.to_crs(4326)
    gdf_zip['longitude'] = gdf_zip.centroid.x
    gdf_zip['latitude'] = gdf_zip.centroid.y

    # Normalize column names
    # If you want to make column names lowercase for consistency
    gdf_zip.columns = [col.lower() for col in gdf_zip.columns]

    # Normalize data types if needed
    # For example, ensuring ZIPCODE is a string, not a number
    #gdf_zip['geometry'] = gdf_zip['geometry'].astype(str)

    # Additional cleaning steps can be added here depending on your specific requirements

    return gdf_zip
    #raise NotImplementedError()

In [79]:
#load_and_clean_zipcodes(ZIPCODE_DATA_FILE)

geodf_zipcode_data = load_and_clean_zipcodes(ZIPCODE_DATA_FILE)
print(type(geodf_zipcode_data))

<class 'geopandas.geodataframe.GeoDataFrame'>


/var/folders/0h/g9zsw1191fl05wqh_8_l7p_w0000gn/T/ipykernel_43282/2287771014.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_zip['longitude'] = gdf_zip.centroid.x
/var/folders/0h/g9zsw1191fl05wqh_8_l7p_w0000gn/T/ipykernel_43282/2287771014.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_zip['latitude'] = gdf_zip.centroid.y


In [14]:
def download_and_clean_311_data():
    import requests

# Your Application Token
    app_token = 'UydgE9GUfZyuG9IpbKml1aKct'

# API endpoints for 311 and tree data
    api_endpoint_311 = 'https://data.cityofnewyork.us/resource/erm2-nwe9.json?$limit=5000'


# Headers for authentication
    headers = {
        'X-App-Token': app_token
    }

# Make a GET request to the 311 data endpoint
    response_311 = requests.get(api_endpoint_311, headers=headers)
    data_311 = response_311.json()

# Assuming you have loaded data into DataFrame
    df = pd.DataFrame(data_311)  # Example for 311 data

# Removing unnecessary columns
    columns_to_keep = ['created_date', 'incident_zip', 'latitude','longitude']  # replace with actual column names
    df = df[columns_to_keep]

# Remove invalid data points
    #df = df[df[':@computed_region_efsh_h5xi'] != null] 
    df = df.dropna()# Replace with your criteria

# Normalize column names
    df.columns = [col.lower().replace(' ', '_') for col in df.columns]

# Convert data types
    #df['date_column'] = pd.to_datetime(df['date_column'])
   # df['numeric_column'] = pd.to_numeric(df['numeric_column'], errors='coerce')

# For geospatial data
    gdf_311 = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
    gdf_311.set_crs("EPSG:4326", inplace=True)  # Replace with current SRID
    return gdf_311
    sampled_df = df.sample(n=10)  # Replace 100 with the number of samples you need
    print(sampled_df)


In [15]:
#download_and_clean_311_data()

,created_date,incident_zip,latitude,longitude,geometry
0,2023-12-02T12:00:00.000,11372,40.756871782232295,-73.87297694367003,POINT (-73.87298 40.75687)
1,2023-12-02T12:00:00.000,11372,40.756822996393886,-73.87354012374385,POINT (-73.87354 40.75682)
2,2023-12-02T01:21:18.000,10025,40.79983820473634,-73.97026715976581,POINT (-73.97027 40.79984)
3,2023-12-02T01:21:15.000,11208,40.68035608925761,-73.88339084745539,POINT (-73.88339 40.68036)
4,2023-12-02T01:20:49.000,10032,40.8330075642874,-73.94041794243994,POINT (-73.94042 40.83301)
...,...,...,...,...,...
4995,2023-12-01T13:10:41.000,11213,40.67127134171268,-73.92733244608434,POINT (-73.92733 40.67127)
4996,2023-12-01T13:10:41.000,11213,40.67127134171268,-73.92733244608434,POINT (-73.92733 40.67127)
4997,2023-12-01T13:10:41.000,11213,40.67127134171268,-73.92733244608434,POINT (-73.92733 40.67127)
4998,2023-12-01T13:10:37.000,11224,40.573494426424446,-73.99283675303543,POINT (-73.99284 40.57349)


In [16]:
def download_and_clean_tree_data():
    app_token = 'UydgE9GUfZyuG9IpbKml1aKct'
    api_endpoint_tree = 'https://data.cityofnewyork.us/resource/5rq2-4hqu.json?$limit=5000'
    headers = {
        'X-App-Token': app_token
    }
    response_tree = requests.get(api_endpoint_tree, headers=headers)
    data_tree = response_tree.json()
    df2 = pd.DataFrame(data_tree)
    columns_to_keep = ['tree_id', 'latitude','longitude','zipcode','spc_common']  # replace with actual column names
    df2 = df2[columns_to_keep]
    df2 = df2.dropna()
    df2.columns = [col.lower().replace(' ', '_') for col in df2.columns]
    gdf_tree = gpd.GeoDataFrame(df2, geometry=gpd.points_from_xy(df2.longitude.astype(float), df2.latitude.astype(float)))
    gdf_tree.set_crs("EPSG:4326", inplace=True)
    return gdf_tree
    sample_tree = gdf.sample(n=10)
    print(sample_tree)
    

In [17]:
#download_and_clean_tree_data()

,tree_id,latitude,longitude,zipcode,spc_common,geometry
0,180683,40.72309177,-73.84421522,11375,red maple,POINT (-73.84422 40.72309)
1,200540,40.79411067,-73.81867946,11357,pin oak,POINT (-73.81868 40.79411)
2,204026,40.71758074,-73.9366077,11211,honeylocust,POINT (-73.93661 40.71758)
3,204337,40.71353749,-73.93445616,11211,honeylocust,POINT (-73.93446 40.71354)
4,189565,40.66677776,-73.97597938,11215,American linden,POINT (-73.97598 40.66678)
...,...,...,...,...,...,...
4995,189566,40.66682867,-73.97608467,11215,London planetree,POINT (-73.97608 40.66683)
4996,196084,40.70452153,-73.82801544,11415,London planetree,POINT (-73.82802 40.70452)
4997,176319,40.59284143,-73.78844287,11692,golden raintree,POINT (-73.78844 40.59284)
4998,172943,40.62945401,-73.99084297,11219,Schumard's oak,POINT (-73.99084 40.62945)


In [30]:
def load_and_clean_zillow_data():
   # Load the data using pandas, as we need to handle both non-spatial and spatial data
    df_zillow = pd.read_csv(ZILLOW_DATA_FILE)

    # Keep only the necessary columns
    columns_to_keep = ["RegionID", "SizeRank", "RegionName", "City", "CountyName", "2023-01-31", "2023-08-31", "2023-09-30"]
    df_zillow = df_zillow[columns_to_keep]

    # Remove rows with missing values in any of the columns
    df_zillow = df_zillow.dropna()
    df_zillow = df_zillow[df_zillow['City']=='New York']

    # Normalize column names to lowercase
    df_zillow.columns = [col.lower() for col in df_zillow.columns]

    # Normalize data types
    # Assuming 'RegionID' and 'SizeRank' should be integers, and dates should be in datetime format
    df_zillow['regionid'] = df_zillow['regionid'].astype(int)
    df_zillow['sizerank'] = df_zillow['sizerank'].astype(int)


    # Convert DataFrame to GeoDataFrame
    gdf_zillow = gpd.GeoDataFrame(df_zillow)

    # Normalize SRID of any geometry if present
    # Example: gdf.set_crs(epsg=YOUR_EPSG_CODE, inplace=True)

    return gdf_zillow

In [31]:
#load_and_clean_zillow_data()

,regionid,sizerank,regionname,city,countyname,2023-01-31,2023-08-31,2023-09-30
4,62093,7,11385,New York,Queens County,2895.699421,3064.476503,3079.585783
6,62019,9,11208,New York,Kings County,2588.030194,2737.547470,2728.733333
13,61807,17,10467,New York,Bronx County,2155.617718,2353.686402,2423.888889
14,62085,18,11373,New York,Queens County,2255.604528,2302.557354,2292.994444
15,62037,19,11226,New York,Kings County,2680.683700,2785.320137,2762.435713
...,...,...,...,...,...,...,...,...
6441,61771,10844,10280,New York,New York County,4059.897856,4602.724208,4657.350427
6550,61773,12178,10282,New York,New York County,7143.347680,7444.236260,7347.458333
6561,62010,12258,11109,New York,Queens County,4159.759749,4490.367074,4529.358974
6644,61620,14713,10006,New York,New York County,3747.416799,4042.831474,4060.096154


In [38]:

geodf_zipcode_data = load_and_clean_zipcodes(ZIPCODE_DATA_FILE)
geodf_311_data = download_and_clean_311_data()
geodf_tree_data = download_and_clean_tree_data()
df_zillow_data = load_and_clean_zillow_data()


    ZIPCODE BLDGZIP        PO_NAME  POPULATION          AREA STATE    COUNTY  \
0     11436       0        Jamaica     18681.0  2.269930e+07    NY    Queens   
1     11213       0       Brooklyn     62426.0  2.963100e+07    NY     Kings   
2     11212       0       Brooklyn     83866.0  4.197210e+07    NY     Kings   
3     11225       0       Brooklyn     56527.0  2.369863e+07    NY     Kings   
4     11218       0       Brooklyn     72280.0  3.686880e+07    NY     Kings   
..      ...     ...            ...         ...           ...   ...       ...   
258   10310       0  Staten Island     25003.0  5.346328e+07    NY  Richmond   
259   11693       0   Far Rockaway     11052.0  3.497516e+06    NY     Kings   
260   11249       0       Brooklyn     28481.0  1.777221e+07    NY     Kings   
261   10162       1       New York         0.0  2.103489e+04    NY  New York   
262   10119       1       New York         0.0  1.263930e+05    NY  New York   

    ST_FIPS CTY_FIPS                   

/var/folders/0h/g9zsw1191fl05wqh_8_l7p_w0000gn/T/ipykernel_43282/1674574668.py:17: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_zip['longitude'] = gdf_zip.centroid.x
/var/folders/0h/g9zsw1191fl05wqh_8_l7p_w0000gn/T/ipykernel_43282/1674574668.py:18: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_zip['latitude'] = gdf_zip.centroid.y
/var/folders/0h/g9zsw1191fl05wqh_8_l7p_w0000gn/T/ipykernel_43282/1674574668.py:26: UserWarning: Geometry column does not contain geometry.
  gdf_zip['geometry'] = gdf_zip['geometry'].astype(str)


In [ ]:
print(type(geodf_311_data))
print(type(geodf_tree_data))

<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'geopandas.geodataframe.GeoDataFrame'>


In [ ]:
# Show basic info about each dataframe
geodf_zipcode_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   geometry  263 non-null    object
dtypes: object(1)
memory usage: 2.2+ KB


In [67]:
# Show first 5 entries about each dataframe
geodf_zipcode_data.head()

,zipcode,geometry,longitude,latitude
0,11436,"POLYGON ((-73.80585 40.68291, -73.80569 40.682...",-73.796503,40.675872
1,11213,"POLYGON ((-73.93740 40.67973, -73.93487 40.679...",-73.936327,40.671156
2,11212,"POLYGON ((-73.90294 40.67084, -73.90223 40.668...",-73.913077,40.662813
3,11225,"POLYGON ((-73.95797 40.67066, -73.95576 40.670...",-73.953960,40.663115
4,11218,"POLYGON ((-73.97208 40.65060, -73.97192 40.650...",-73.976451,40.643320


In [ ]:
geodf_311_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 4936 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   created_date  4936 non-null   object  
 1   incident_zip  4936 non-null   object  
 2   latitude      4936 non-null   object  
 3   longitude     4936 non-null   object  
 4   geometry      4936 non-null   geometry
dtypes: geometry(1), object(4)
memory usage: 231.4+ KB


In [ ]:
geodf_311_data.head()

,created_date,incident_zip,latitude,longitude,geometry
0,2023-12-02T12:00:00.000,11372,40.756822996393886,-73.87354012374385,POINT (-73.87354 40.75682)
1,2023-12-02T12:00:00.000,11372,40.756871782232295,-73.87297694367003,POINT (-73.87298 40.75687)
2,2023-12-02T01:21:18.000,10025,40.79983820473634,-73.97026715976581,POINT (-73.97027 40.79984)
3,2023-12-02T01:21:15.000,11208,40.68035608925761,-73.88339084745539,POINT (-73.88339 40.68036)
4,2023-12-02T01:20:49.000,10032,40.8330075642874,-73.94041794243994,POINT (-73.94042 40.83301)


In [ ]:
geodf_tree_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 4810 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   latitude    4810 non-null   object  
 1   longitude   4810 non-null   object  
 2   zipcode     4810 non-null   object  
 3   spc_common  4810 non-null   object  
 4   geometry    4810 non-null   geometry
dtypes: geometry(1), object(4)
memory usage: 225.5+ KB


In [ ]:
geodf_tree_data.head()

,latitude,longitude,zipcode,spc_common,geometry
0,40.72309177,-73.84421522,11375,red maple,POINT (-73.84422 40.72309)
1,40.79411067,-73.81867946,11357,pin oak,POINT (-73.81868 40.79411)
2,40.71758074,-73.9366077,11211,honeylocust,POINT (-73.93661 40.71758)
3,40.71353749,-73.93445616,11211,honeylocust,POINT (-73.93446 40.71354)
4,40.66677776,-73.97597938,11215,American linden,POINT (-73.97598 40.66678)


In [ ]:
df_zillow_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 121 entries, 4 to 6721
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   regionid    121 non-null    int64  
 1   sizerank    121 non-null    int64  
 2   regionname  121 non-null    int64  
 3   statename   121 non-null    object 
 4   city        121 non-null    object 
 5   metro       121 non-null    object 
 6   countyname  121 non-null    object 
 7   2023-01-31  121 non-null    float64
 8   2023-08-31  121 non-null    float64
dtypes: float64(2), int64(3), object(4)
memory usage: 9.5+ KB


In [ ]:
df_zillow_data.head()

,regionid,sizerank,regionname,statename,city,metro,countyname,2023-01-31,2023-08-31
4,62093,7,11385,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,2895.699421,3064.476503
6,62019,9,11208,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Kings County,2588.030194,2737.547470
13,61807,17,10467,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Bronx County,2155.617718,2353.686402
14,62085,18,11373,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,2255.604528,2302.557354
15,62037,19,11226,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Kings County,2680.683700,2785.320137


## Part 2: Storing Data

In [21]:
!createdb IE4501_Project_DB

createdb: error: database creation failed: ERROR:  database "IE4501_Project_DB" already exists


In [22]:
!psql --dbname IE4501_Project_DB -c 'CREATE EXTENSION postgis;'

ERROR:  extension "postgis" already exists


In [23]:
engine = db.create_engine(DB_URL)

### Creating Tables


These are just a couple of options to creating your tables; you can use one or the other, a different method, or a combination.

In [ ]:
engine = db.create_engine(DB_URL)

#### Option 1: SQL

In [32]:
# if using SQL (as opposed to SQLAlchemy), define the SQL statements to create your 4 tables
ZIPCODE_SCHEMA = """
    CREATE TABLE IF NOT EXISTS geodf_zipcode_data (
        id SERIAL PRIMARY KEY,
        zipcode VARCHAR(10),
        geometry GEOMETRY,
        longitude FLOAT,
        latitude FLOAT
    );
    """

NYC_311_SCHEMA = """
    CREATE TABLE IF NOT EXISTS geodf_311_data (
        id SERIAL PRIMARY KEY,
        created_date DATE,
        incident_zip VARCHAR(10),
        latitude FLOAT,
        longitude FLOAT,
        geometry GEOMETRY
    );
    """

NYC_TREE_SCHEMA = """
    CREATE TABLE IF NOT EXISTS geodf_tree_data (
        id SERIAL PRIMARY KEY,
        tree_id INTEGER,
        latitude FLOAT,
        longitude FLOAT,
        zipcode VARCHAR(10),
        spc_common VARCHAR(100),
        geometry GEOMETRY
    );
    """

ZILLOW_SCHEMA = """
    CREATE TABLE IF NOT EXISTS df_zillow_data (
        RegionID INTEGER PRIMARY KEY,
        SizeRank INT,
        RegionName VARCHAR(100),
        City VARCHAR(100),
        CountyName VARCHAR(100),
        "2023-01-31" FLOAT,
        "2023-08-31" FLOAT,
        "2023-09-30" FLOAT
    );
    """

In [33]:
# create that required schema.sql file
with open(DB_SCHEMA_FILE, "w") as f:
    f.write(ZIPCODE_SCHEMA)
    f.write(NYC_311_SCHEMA)
    f.write(NYC_TREE_SCHEMA)
    f.write(ZILLOW_SCHEMA)

In [34]:
conn = psycopg2.connect("dbname='IE4501_Project_DB' user='postgres'")
conn.autocommit = True
cursor = conn.cursor()

# Execute the table creation commands
cursor.execute(ZIPCODE_SCHEMA)
cursor.execute(NYC_311_SCHEMA)
cursor.execute(NYC_TREE_SCHEMA)
cursor.execute(ZILLOW_SCHEMA)

# Close the cursor and connection
cursor.close()
conn.close()

In [35]:
# If using SQL (as opposed to SQLAlchemy), execute the schema files to create tables
with engine.connect() as connection:
    pass

#### Option 2: SQLAlchemy

In [ ]:
Base = declarative_base()

class Tree(Base):
    __tablename__ = "trees"

    ...


In [ ]:
Base.metadata.create_all(engine)

### Add Data to Database

These are just a couple of options to write data to your tables; you can use one or the other, a different method, or a combination.

#### Option 1: SQL

In [89]:
def write_dataframes_to_table(tablename_to_dataframe):
    # write INSERT statements or use pandas/geopandas to write SQL
    engine = create_engine(DB_URL)
    #for tablename, dataframe in tablename_to_dataframe.items():
    # Iterate over the dictionary and write each dataframe to its respective table
        #if 'geometry' in dataframe.columns:
                # Convert geometry to WKTElement
                #dataframe['geometry'] = dataframe['geometry'].apply(lambda x: WKTElement(x.wkt, srid=4326))

            # Write the dataframe to the SQL table
        #dataframe.to_sql(tablename, engine, if_exists='append', index=False, dtype={'geometry': Geometry('GEOMETRY', srid=4326)})

    # Close the engine connection
    #engine.dispose()
    for tablename, dataframe in tablename_to_dataframe.items():
        if 'geometry' in dataframe.columns:
            # Ensure the column is in the correct format
            if not isinstance(dataframe.iloc[0]['geometry'], WKTElement):
                dataframe['geometry'] = dataframe['geometry'].apply(lambda x: WKTElement(x.wkt, srid=4326))
                dataframe.to_sql(tablename, engine, if_exists='replace', index=False, dtype={'geometry': Geometry('GEOMETRY', srid=4326)})
        else: 
            dataframe.to_sql(tablename, engine, if_exists='replace', index = False)
    engine.dispose()

In [90]:
tablename_to_dataframe = {
    "zipcodes": geodf_zipcode_data,
    "complaints": geodf_311_data,
    "trees": geodf_tree_data,
    "rents": df_zillow_data,
}

In [91]:
write_dataframes_to_table(tablename_to_dataframe)

#### Option 2: SQLAlchemy

In [ ]:
Session = db.orm.sessionmaker(bind=engine)
session = Session()

In [ ]:
for row in geodf_tree_data.iterrows():
    tree = Tree(...)
    session.add(tree)

In [ ]:
session.commit()

## Part 3: Understanding the Data

### Query 1

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

In [ ]:
QUERY_1_FILENAME = QUERY_DIR / "FILL_ME_IN"

QUERY_1 = """
FILL_ME_IN
"""

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_1))
    for row in result:
        print(row)

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query your database for the data needed.
    # You can put the data queried into a pandas/geopandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)